In [2]:
import os

In [3]:
%pwd

'd:\\git_folder\\MLoperations-project-with-mlflow\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'd:\\git_folder\\MLoperations-project-with-mlflow'

In [18]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/Dhanush-R5/MLoperations-project-with-mlflow.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "Dhanush-R5"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "PfpmJcTM5DJS7@9"

In [19]:
from dataclasses import dataclass
from pathlib import Path

In [20]:
@dataclass(frozen=True)
class ModelEvaluationConfig:
   root_dir: Path
   test_data_path: Path
   model_path: Path
   all_params: dict
   metric_file_name: Path
   target_column: str
   mlflow_uri: str

In [21]:
from MLproject.constants import *
from MLproject.utils.common import read_yaml, create_directories, save_json

In [22]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir = config.root_dir, 
            test_data_path = config.test_data_path,
            model_path = config.model_path,
            all_params = params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri = "https://dagshub.com/Dhanush-R5/MLoperations-project-with-mlflow.mlflow",

        )
        return model_evaluation_config

In [23]:
import os
import pandas as pd 
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow 
import mlflow.sklearn
import numpy as np 
import joblib 

In [24]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self,actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    
    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme


        with mlflow.start_run():

            predicted_qualities = model.predict(test_x)

            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)
            
            # Saving metrics as local
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)

             # Model registry does not work with file store
            if tracking_url_type_store != "file":
                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticnetModel")
            else:
                mlflow.sklearn.log_model(model, "model")

In [25]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2024-06-22 00:05:10,023: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-06-22 00:05:10,025: INFO: common: yaml file: params.yaml loaded successfully]
[2024-06-22 00:05:10,037: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-06-22 00:05:10,042: INFO: common: created directory at: artifacts]
[2024-06-22 00:05:10,046: INFO: common: created directory at: artifacts/model_evaluation]
[2024-06-22 00:05:10,257: WARNING: connectionpool: Retrying (Retry(total=4, connect=5, read=4, redirect=5, status=5)) after connection broken by 'RemoteDisconnected('Remote end closed connection without response')': /Dhanush-R5/MLoperations-project-with-mlflow.mlflow/api/2.0/mlflow/runs/create]
[2024-06-22 00:05:12,528: INFO: common: json file saved at: artifacts\model_evaluation\metrices.json]


Successfully registered model 'ElasticnetModel'.
2024/06/22 00:05:30 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: ElasticnetModel, version 1
Created version '1' of model 'ElasticnetModel'.
